In [2]:
import os
from netml.pparser.parser import PCAP
import pprint
from scapy.all import *
from scapy.layers.inet import *
from tqdm import tqdm

import tensorflow as tf


In [11]:
dataDir = "/code/2.Malwares/"
targetFile = "Zeus.pcap"
destination = dataDir + "out"
tsharkPath="/usr/bin/tshark"
packetSizeInBytes = 100
packetsPerFlow = 2
tfrecorsFolder = dataDir + 'tfrecords/'
testFolder = dataDir + 'test/'
validationFolder = dataDir + 'validation/'

nb_shards = 10
arrOfTfrecordsWriter = [] # un objet criter par shard

In [4]:
cats = ["Benign","Malware"] #Catégorie
listofProcessedFiles = []
training_data = []

In [ ]:
for i in range(nb_shards):
    arrOfTfrecordsWriter.append(tf.io.TFRecordWriter(path=tfrecorsFolder + 'shard_' + str(i) + '.tfrecords'))

In [ ]:
def getLabelForFile(filename):
    filename.replace('.pcap','')
    if filename.find('-') != -1:
        filename = filename.split('-')[0]
    for index,processedFile in enumerate(listofProcessedFiles):
        if processedFile == filename:
            return index
    listofProcessedFiles.append(filename)
    return len(listofProcessedFiles)
    
#pprint.pprint(listofProcessedFiles)
#label = getLabelForFile('tes-test')
#print(label)

In [ ]:
def _bytes_feature(value):# peut-être un tableau
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()  # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [ ]:
def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
# Create the features dictionary.
def packet_example(packetArray, label):
    feature = {
        'label': _int64_feature(label),
        'packet1': _bytes_feature(packetArray[0]),
        'packet2': _bytes_feature(packetArray[1]),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

In [ ]:
 def processPackets(packets):
    newFlow = []
    for i,packet in enumerate(packets):
        packetHex = linehexdump(packet.payload,onlyhex=1,dump=True) # dump packet to hex 01 DE 
        packetHexnoSpace = packetHex.replace(' ','').lower() #to hex 01de
        packetBytes = bytes.fromhex(packetHexnoSpace) # to bytestring
        if len(packetBytes) < packetSizeInBytes: # ajuster les packets à une taille de 100 bytes 
            packetPadded = packetBytes.ljust(packetSizeInBytes,b'\0') #padding
        else:
            packetPadded = packetBytes[:packetSizeInBytes] # couper les paquet trop grand
        if i >= packetsPerFlow:
            break
        newFlow.append(packetPadded)
    return newFlow


In [ ]:
import random
for cat in cats: # pour chaque dossier Benign, Malware
    path = os.path.join(dataDir,cat)
    for filePcap in os.listdir(path): #parcours les fichiers pour chaque dossier
        pprint.pprint(path)
        pprint.pprint(filePcap)

        parsedPcap = PCAP(
            os.path.join(path,filePcap),
            flow_ptks_thres=packetsPerFlow,
            verbose=3
        )
        parsedPcap.pcap2flows(interval=1000)
        for flow in tqdm(parsedPcap.flows): #(val,182.168.1.15,,)(<Ether
            flow = flow[1:] # retirer le premier élément car inutile
            for packets in flow:
                writer = random.choice(arrOfTfrecordsWriter)
                tf_example = packet_example(processPackets(packets), getLabelForFile(filePcap))
                writer.write(tf_example.SerializeToString())



In [13]:
tfrecordsPath = os.path.join(tfrecorsFolder)
listtfRecordsFile = os.listdir(tfrecordsPath)
validationFiles = random.sample(listtfRecordsFile,2)
pprint.pprint(validationFiles)
os.rename(tfrecordsPath + validationFiles[0],validationFolder + validationFiles[0])
os.rename(tfrecordsPath + validationFiles[1],testFolder + validationFiles[1])

['shard_2.tfrecords', 'shard_6.tfrecords']
